In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("ultimate2.csv")

In [3]:
df.head()

,Unnamed: 0,City1,City1_long,City1_lat,City2,City2_long,City2_lat,City3,City3_long,City3_lat,Prompt,Response,correct,Closer_City
0,0,"Goldsboro, NC",-77.9720,35.3778,"St. Cloud, MN",-94.1718,45.5339,"Stamford, CT",-73.5583,41.1035,Is Goldsboro NC closer to St. Cloud MN or Stam...,"Stamford, CT",True,"Stamford, CT"
1,1,"Woodbury, MN",-92.9230,44.9056,"Medford, MA",-71.1087,42.4234,"Des Plaines, IL",-87.9009,42.0345,Is Woodbury MN closer to Medford MA or Des Pla...,"Des Plaines, IL",True,"Des Plaines, IL"
2,2,"Muncie, IN",-85.3950,40.1989,"Roswell, GA",-84.3513,34.0391,"Houma, LA",-90.7058,29.5799,Is Muncie IN closer to Roswell GA or Houma LA?,"Roswell, GA",True,"Roswell, GA"
3,3,"Glendora, CA",-117.8468,34.1449,"Lafayette, LA",-92.0323,30.2084,"Dublin, CA",-121.8963,37.7161,Is Glendora CA closer to Lafayette LA or Dubli...,"Dublin, CA",True,"Dublin, CA"
4,4,"San Juan, PR",-66.0636,18.4037,"Rome, GA",-85.1862,34.2661,"Federal Way, WA",-122.3358,47.3091,Is San Juan PR closer to Rome GA or Federal Wa...,"Rome, GA",True,"Rome, GA"


In [4]:
df = df[["City1", "City2", "City3", "Closer_City", "Response"]]

In [5]:
df['y'] = df.apply(lambda row: 1 if row['Response'] == row['City2'] else 0, axis=1)
df = df[['City1', 'City2', 'City3', 'y']]
df.head()

,City1,City2,City3,y
0,"Goldsboro, NC","St. Cloud, MN","Stamford, CT",0
1,"Woodbury, MN","Medford, MA","Des Plaines, IL",0
2,"Muncie, IN","Roswell, GA","Houma, LA",1
3,"Glendora, CA","Lafayette, LA","Dublin, CA",0
4,"San Juan, PR","Rome, GA","Federal Way, WA",1


In [8]:
from transformers import GPT2Tokenizer, GPT2Model
import torch
import torchvision
import numpy as np

In [9]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

# Tokenize the city names using the GPT-2 tokenizer
df['City1'] = df['City1'].apply(lambda x: tokenizer.encode(x))
df['City2'] = df['City2'].apply(lambda x: tokenizer.encode(x))
df['City3'] = df['City3'].apply(lambda x: tokenizer.encode(x))

df.head()

,City1,City2,City3,y
0,"[38, 10119, 21513, 11, 8823]","[1273, 13, 10130, 11, 29060]","[1273, 321, 3841, 11, 16356]",0
1,"[22911, 10711, 11, 29060]","[9921, 3841, 11, 8779]","[5960, 28847, 274, 11, 14639]",0
2,"[44, 19524, 494, 11, 3268]","[35740, 4053, 11, 14545]","[39, 280, 2611, 11, 9131]",1
3,"[9861, 437, 5799, 11, 7257]","[43, 1878, 27067, 11, 9131]","[37590, 2815, 11, 7257]",0
4,"[15017, 16852, 11, 4810]","[49, 462, 11, 14545]","[24099, 6378, 11, 16400]",1


In [10]:
model = GPT2Model.from_pretrained('gpt2')
def get_gpt2_embedding(tokens):
    with torch.no_grad():
        inputs = torch.tensor(tokens).unsqueeze(0)
        outputs = model(inputs)
        embedding = outputs.last_hidden_state.mean(dim=1).squeeze().numpy()
    return embedding
df['City1'] = df['City1'].apply(get_gpt2_embedding)
df['City2'] = df['City2'].apply(get_gpt2_embedding)
df['City3'] = df['City3'].apply(get_gpt2_embedding)
df.head()

,City1,City2,City3,y
0,"[-0.13778201, -0.2620504, -0.24149883, 0.27540...","[0.17534807, -0.22879699, -0.1766169, 0.032793...","[0.018141117, -0.14230137, -0.3901435, 0.23501...",0
1,"[-0.058927465, -0.14931548, -0.04336655, 0.085...","[0.1388114, -0.21189696, -0.12958553, -0.03190...","[-0.18448898, -0.41184106, -0.11619693, 0.1268...",0
2,"[-0.033003174, -0.2621144, -0.04794469, -0.188...","[0.055776983, -0.26947463, 0.04635828, -0.0559...","[-0.0042285444, -0.25156856, 0.010114083, -0.1...",1
3,"[-0.08724886, -0.07307617, -0.06942039, 0.0446...","[0.14595151, -0.33073527, 0.0983743, -0.190553...","[-0.23472063, -0.22974622, -0.24161044, -0.034...",0
4,"[-0.093845576, -0.40075433, 0.021676645, 0.039...","[-0.2648417, -0.5007277, -0.030107122, -0.0196...","[0.034900166, -0.2295934, -0.4087085, 0.039263...",1


In [ ]:
df.to_csv("gpt2_embeddings.csv")